In [ ]:
import numpy as np
import math
from numpy.random import beta
import time

# to compute exp of each cell in a matrix
exp = np.vectorize(math.exp)


def MV(Psi):
    """
    The majority voting method.
    """
    return [np.bincount([y[1] for y in x]) / (0.0 + len(x)) for x in Psi]


def EM(N, M, Psi, inv_Psi):
    """
    The expectation maximization method (EM) from Dong et al., 2013.
    """
    # convergence eps
    eps = 0.0001

    # init accuracies
    A = [0.8]*N
    while True:
        # E-step
        p = [[0, 0] for x in range(M)]
        for obj in range(M):
            C = [0]*2
            for s, val in Psi[obj]:
                C[val] += math.log(A[s])
                C[not val] += math.log(1-A[s])
            p[obj][1] = math.exp(C[1])/(math.exp(C[0])+math.exp(C[1]))
            p[obj][0] = math.exp(C[0]) / (math.exp(C[0]) + math.exp(C[1]))

        # M-step
        A_new = [np.average([p[y[0]][y[1]] for y in x]) for x in inv_Psi]

        # convergence check
        if sum(abs(np.subtract(A, A_new))) < eps:
            A = A_new
            break
        else:
            A = A_new

    return A, p


def log_likelihood(Psi, A, p):
    """
    Computes the log likelihood of the Psi using A and p.
    """
    res = 0
    for obj_id in range(M):
        for source_id, value_id in Psi[obj_id]:
            if value_id == 1:
                res += math.log(A[source_id] * p[obj_id][value_id])
            else:
                res += math.log((1 - A[source_id]) * (1 - p[obj_id][value_id]))
    return res


def random_log_likelihood(N, M, Psi):
    """
    Searches for the max log likelihood at random.
    """
    # number of attempts
    N_iter = 10

    max_log_likelihood = -10000000.0
    bf_A = []
    bf_p = []
    for i in range(N_iter):
        A = np.random.uniform(0.8, 1.0, N)
        p = [[1 - x, x] for x in np.random.uniform(0, 1, M)]
        cur_ll = log_likelihood(Psi, A, p)
        if cur_ll > max_log_likelihood:
            max_log_likelihood = cur_ll
            bf_A = A
            bf_p = p

    return bf_A, bf_p


def mcmc(N, M, Psi, inv_Psi):
    """
    MCMC for log-likelihood maximum search.
    """
    N_iter = 15
    burnin = 5
    thin = 2

    # random init
    A = np.random.uniform(0.8, 1.0, N)

    # MCMC sampling
    sample_size = 0.0
    mcmc_p = [[0.0, 0.0] for x in range(M)]
    for _iter in range(N_iter):
        # update objects
        p = [[0.0, 0.0] for x in range(M)]
        for obj in range(M):
            C = [0.0]*2
            for s, val in Psi[obj]:
                C[val] += math.log(A[s])
                C[not val] += math.log(1-A[s])
            p[obj][1] = math.exp(C[1])/(math.exp(C[0])+math.exp(C[1]))
            p[obj][0] = math.exp(C[0]) / (math.exp(C[0]) + math.exp(C[1]))
        O = [1 if np.random.rand() < p[i][1] else 0 for i in range(M)]

        # update sources
        for source_id in range(N):
            beta_0 = 0
            beta_1 = 0
            for obj, val in inv_Psi[source_id]:
                if val == O[obj]:
                    beta_0 += 1
                else:
                    beta_1 += 1
            A[source_id] = beta(beta_0 + 4, beta_1 + 1)

        #print(A, O)
        if _iter > burnin and _iter % thin == 0:
            sample_size += 1
            for obj in range(M):
                mcmc_p[obj][O[obj]] += 1

    # mcmc output
    # fix: if there are zero counts change them to tiny amounts
    for p in mcmc_p:
        if p[0] == 0:
            p[0] += 0.0001
            p[1] -= 0.0001
        elif p[1] == 0:
            p[0] -= 0.0001
            p[1] += 0.0001
    mcmc_p = [[p[0]/sample_size, p[1]/sample_size] for p in mcmc_p]
    mcmc_A = [0]*N
    for s in range(N):
        for obj, val in inv_Psi[s]:
            # TODO take advantage of priors (as in Zhao et al., 2012)
            mcmc_A[s] += mcmc_p[obj][val]
        mcmc_A[s] /= (0.0+len(inv_Psi[s]))

    return mcmc_A, mcmc_p

# number of sources
N = 20
# number of objects
M = 1000000
# synthetically generated observations
density = 0.7
accuracy = 0.8
Psi = [[(s, int(np.random.rand() < density)) for s in range(N) if np.random.rand() < accuracy] for obj in range(M)]
# compute inverted Psi (indexing)
inv_Psi = [[] for s in range(N)]
for obj in range(M):
    for s, val in Psi[obj]:
        inv_Psi[s].append( (obj, val) )

start = time.time()
mv_p = MV(Psi)
mv_t = time.time() - start

start = time.time()
em_A, em_p = EM(N, M, Psi, inv_Psi)
em_t = time.time() - start

start = time.time()
bf_A, bf_p = random_log_likelihood(N, M, Psi)
bf_t = time.time() - start

start = time.time()
mcmc_A, mcmc_p = mcmc(N, M, Psi, inv_Psi)
mcmc_t = time.time() - start

print('MV   Pr: {:1.3f}\tlog-likelihood: N/A  \ttime: {:5.1f}'.format(np.average([x[1] for x in mv_p]), mv_t))
print('EM   Pr: {:1.3f}\tlog-likelihood: {:3.2f}\ttime: {:5.1f}'.format(np.average([x[1] for x in em_p]), log_likelihood(Psi, em_A, em_p), em_t))
print('RND  Pr: {:1.3f}\tlog-likelihood: {:3.2f}\ttime: {:5.1f}'.format(np.average([x[1] for x in bf_p]), log_likelihood(Psi, bf_A, bf_p), bf_t))
print('MCMC Pr: {:1.3f}\tlog-likelihood: {:3.2f}\ttime: {:5.1f}'.format(np.average([x[1] for x in mcmc_p]), log_likelihood(Psi, mcmc_A, mcmc_p), mcmc_t))